Сейчас будем решать задачу классификации текста - вещь приближенная к реальности. 

как нам это делать?

давайте представим задачу попроще - задача классификации отзыва на каком нибудь интернет магазине - люди приходят и пишут комментарии про то как им товар. манеджеру очень-очень хочется чтобы все были довольны, и если кто-то расстроен, то он отправит кого-нибудь из поддержки на помощь. 

Как бы помочь менеджеру выявлять грустных клиентов по их отзывам? Классифицировать их на два класса - хороший отзыв или плохой!

_самый простой способ это делать - просто закостылить if-чиками программу которая будет ходить по всем отзывам и смотреть, есть ли в них слова из группы {огонь, рад, счастлив, устраивает, замечательный и тд} и из группы {расстроен, плохой, ненавижу, грустно, печально, итд} и в зависимости от этого классифицировать отзывы на две группы._

например отзыв: "я рад что купил ваш товар потому что без него не могу теперь" отнесется к положительному классу, а отзыв "больше никогда у вас ничего не куплю, последний товар был плохим и я очень расстроен вашим сервисом" - к негативному отзыву.

____

но тут есть две проблемы:

1) нам сложно выделять все позитивные и негативные слова, все не перечислить, нам надо писать код и дать решение, а не составлять словарь слов с различными окрасками.

2) слова из позитивной группы могут встречаться в негативном отзыве и наоборот, может быть что 3 слова из позитивной группы слов встретились в отзыве (рад, нормально, хорошо), а из негативной только два (плохо, не буду), и не всегда ясно что с таким делать - всегда можно подобрать такой кейс где надо относить такой пример к негативному, а не позитивному, потму что разные слова могут вносить разный вклад в результат (какие то слова более ярко окрашивают "настроение" предложения)


Призовем на помощь машинное обучение!


что мы можем сделать? давайте попросим менеджера разметить по 300-500-1000 отзывов на плохие и хорошие категории, и обучим на этом модель, чтобы она сама подбирала веса для слов.

Скажем, что у каждого слова есть вес $\Large{w}$ - позитивный или негативный (например 100 или -1.234), пусть машинное обучение само подбирает важность и окрас каждого слова и оценит вклад слова в результат. 

Как будет выглядеть наш классификатор?
Возьмем логистическую регрессию 

$$P(cat=positive) = \sigma{(< X, w>)} $$

где $\sigma(x) =  \frac{\mathrm{1} }{\mathrm{1} + e^{-x} } $

X - бинарный вектор, в котором 1 - значит что такое то слово присутствует, а 0 - что отсутствует. 

Тогда идея классификатора в следующем - подберем и сложим веса для различных слов которые встречаются в отзыве, посмотрим вероятности и предскажем к какому классу это относится! Круто.



А теперь давайте решать задачу классификации объявлений одной известной интернет доски.

In [62]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
data = pd.read_csv('train.csv')

In [64]:
data.head()

,item_id,title,description,price,category_id
0,312862,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,339243,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,6677,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,190544,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,372595,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


> нам надо представить каждое объявление в виде набора признаков. с числом price - понятно, это и так признак объявления. однако наши классификаторы не умеют просто так работать с текстом. надо представить текст в виде признаков. какой самый простой способ векторизовать текст?

> представить текст в виде вектора - размерностью число возможных слов, и заполнять значениями сколько раз каждое из слов встретилось в тексте. примерно так же как на картинке ниже

![](https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/assets/feml_0405.png)

в sklearn для этого есть CountVectorizer

In [65]:
title_vectorizer = CountVectorizer(max_features=1000, binary=True)
title_features = title_vectorizer.fit_transform(data.title)

descr_vectorizer = CountVectorizer(max_features=1000, binary=True)
description_features = descr_vectorizer.fit_transform(data.description)

features = sp.hstack([title_features, description_features, data[['price']]])

In [66]:
Xtrain, Xval, ytrain, yval = train_test_split(features, data.category_id, 
                                              random_state=241, test_size=0.33)

In [67]:
%%time

clf = LogisticRegression(C=100)
clf.fit(Xtrain, ytrain)
y_pred = clf.predict(Xval)

CPU times: user 16min 1s, sys: 1.27 s, total: 16min 3s
Wall time: 16min 11s


In [68]:
accuracy_score(yval, y_pred)

0.6193855238784975

##### Предскажем тест

In [69]:
test = pd.read_csv('test.csv')

In [70]:
title_features = title_vectorizer.transform(test.title)
description_features = descr_vectorizer.transform(test.description)


features = sp.hstack([title_features, description_features, test[['price']]])

In [71]:
features.shape

(54361, 2001)

In [72]:
y_pred_test = clf.predict(features)

##### Сохраним в csv

In [73]:
pd.DataFrame({'Id': test.item_id, 'Category':y_pred_test}).to_csv('my_pred.csv', index=None)

Что можно сделать еще?

> Какие недостатки решения выше у нас есть?


> давайте помнить что модель - глупая, она умеет разделять только точки. Хорошо бы нам самим передать ей эти точки так, чтобы глупая модель сама смогла хорошо разделить наши данные. наверное, разные формы слов хорошо было бы представить одним? например "хорошая" и "хороший" - это в принципе про одно качество, а "стул" и "стулья" про один предмет. 


> попробуйте tf-idf, хеширование признаков, что можно сделать с непрерывным признаком, удаление мусорных слов, удаление частотных слов, стемминг или лемматизация

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

стемминг или лемматизация:

https://pymorphy2.readthedocs.io/en/0.2/user/index.html

https://tech.yandex.ru/mystem/

Пишите вопросы в специальный канал, правила игры вы знаете